## Get users associated with a given repo

In [1]:
import pandas as pd
import numpy as np
import helper as h

from google.cloud import bigquery
client = bigquery.Client()

from github3 import login
password = h.get_gh_credentials()
gh = login('bps10', password=password)

In [99]:
def get_contributors(gh, owner, repo):
    repo = gh.repository('numpy', 'numpy')
    return [[contributor.login, contributor.contributions] for contributor in repo.contributors()]


In [100]:
contrib = get_contributors(gh, 'numpy', 'numpy')

In [109]:
repo = gh.repository('numpy', 'numpy')

In [56]:
user = gh.user('bps10')

In [60]:
user_events = user.events(number=-1, public=True)

In [62]:
for event in user_events:
    print(event.type)

PushEvent
PushEvent
CreateEvent
CreateEvent
PushEvent
PushEvent
PushEvent
PushEvent
PublicEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
CreateEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
PushEvent
MemberEvent
PushEvent
PushEvent
PushEvent
MemberEvent
PushEvent


In [86]:
user1='bps10'
user2='hadley'
# Set use_legacy_sql to True to use legacy SQL syntax.
job_config = bigquery.QueryJobConfig()
job_config.use_legacy_sql = True

query = (
"""SELECT  
  actor.login,
  COUNT(type) as event_count, 
  MAX(created_at) as last_event, 
  MIN(created_at) as first_event,
  SUM(CASE WHEN type = 'CommitCommentEvent' then 1 else 0 end) as CommitCommentEvent_count,
  SUM(CASE WHEN type = 'CreateEvent' then 1 else 0 end) as CreateEvent_count,
  SUM(CASE WHEN type = 'DeleteEvent' then 1 else 0 end) as DeleteEvent_count,
  SUM(CASE WHEN type = 'ForkEvent' then 1 else 0 end) as ForkEvent_count,
  SUM(CASE WHEN type = 'GollumEvent' then 1 else 0 end) as GollumEvent_count,
  SUM(CASE WHEN type = 'IssueCommentEvent' then 1 else 0 end) as IssueCommentEvent_count,
  SUM(CASE WHEN type = 'IssuesEvent' then 1 else 0 end) as IssuesEvent_count,
  SUM(CASE WHEN type = 'MemberEvent' then 1 else 0 end) as MemberEvent_count,
  SUM(CASE WHEN type = 'PublicEvent' then 1 else 0 end) as PublicEvent_count,
  SUM(CASE WHEN type = 'PullRequestEvent' then 1 else 0 end) as PullRequestEvent_count,
  SUM(CASE WHEN type = 'PullRequestReviewCommentEvent' then 1 else 0 end) as PullRequestReviewCommentEvent_count,
  SUM(CASE WHEN type = 'PushEvent' then 1 else 0 end) as PushEvent_count,
  SUM(CASE WHEN type = 'ReleaseEvent' then 1 else 0 end) as ReleaseEvent_count,
  SUM(CASE WHEN type = 'WatchEvent' then 1 else 0 end) as WatchEvent_count,

  FROM 
  [githubarchive:year.2018]
WHERE (actor.login = '""" + user1 + """' OR actor.login = '""" + user2 + """' AND public)
GROUP by actor.login;

"""
    
)

#results_df = gbq.read_gbq(query_job, project_id=project_id, private_key='path_to_my.json')

df = client.query(query, location="US", job_config=job_config).to_dataframe()
df

,actor_login,event_count,last_event,first_event,CommitCommentEvent_count,CreateEvent_count,DeleteEvent_count,ForkEvent_count,GollumEvent_count,IssueCommentEvent_count,IssuesEvent_count,MemberEvent_count,PublicEvent_count,PullRequestEvent_count,PullRequestReviewCommentEvent_count,PushEvent_count,ReleaseEvent_count,WatchEvent_count
0,bps10,163,2019-01-04 21:58:39+00:00,2018-02-06 21:26:44+00:00,0,13,0,3,9,0,0,2,1,25,0,109,0,1
1,hadley,10734,2019-01-05 23:18:47+00:00,2018-02-01 04:49:54+00:00,50,155,58,8,0,3770,2088,11,1,988,1134,2441,30,0


In [87]:
query

"SELECT  \n  actor.login,\n  COUNT(type) as event_count, \n  MAX(created_at) as last_event, \n  MIN(created_at) as first_event,\n  SUM(CASE WHEN type = 'CommitCommentEvent' then 1 else 0 end) as CommitCommentEvent_count,\n  SUM(CASE WHEN type = 'CreateEvent' then 1 else 0 end) as CreateEvent_count,\n  SUM(CASE WHEN type = 'DeleteEvent' then 1 else 0 end) as DeleteEvent_count,\n  SUM(CASE WHEN type = 'ForkEvent' then 1 else 0 end) as ForkEvent_count,\n  SUM(CASE WHEN type = 'GollumEvent' then 1 else 0 end) as GollumEvent_count,\n  SUM(CASE WHEN type = 'IssueCommentEvent' then 1 else 0 end) as IssueCommentEvent_count,\n  SUM(CASE WHEN type = 'IssuesEvent' then 1 else 0 end) as IssuesEvent_count,\n  SUM(CASE WHEN type = 'MemberEvent' then 1 else 0 end) as MemberEvent_count,\n  SUM(CASE WHEN type = 'PublicEvent' then 1 else 0 end) as PublicEvent_count,\n  SUM(CASE WHEN type = 'PullRequestEvent' then 1 else 0 end) as PullRequestEvent_count,\n  SUM(CASE WHEN type = 'PullRequestReviewCommentE

In [138]:
def create_query_from_list(user_list):
    txt = (
    """SELECT  
      actor.login,
      COUNT(type) as event_count, 
      MAX(created_at) as last_event, 
      MIN(created_at) as first_event,
      SUM(CASE WHEN type = 'CommitCommentEvent' then 1 else 0 end) as CommitCommentEvent_count,
      SUM(CASE WHEN type = 'CreateEvent' then 1 else 0 end) as CreateEvent_count,
      SUM(CASE WHEN type = 'DeleteEvent' then 1 else 0 end) as DeleteEvent_count,
      SUM(CASE WHEN type = 'ForkEvent' then 1 else 0 end) as ForkEvent_count,
      SUM(CASE WHEN type = 'GollumEvent' then 1 else 0 end) as GollumEvent_count,
      SUM(CASE WHEN type = 'IssueCommentEvent' then 1 else 0 end) as IssueCommentEvent_count,
      SUM(CASE WHEN type = 'IssuesEvent' then 1 else 0 end) as IssuesEvent_count,
      SUM(CASE WHEN type = 'MemberEvent' then 1 else 0 end) as MemberEvent_count,
      SUM(CASE WHEN type = 'PublicEvent' then 1 else 0 end) as PublicEvent_count,
      SUM(CASE WHEN type = 'PullRequestEvent' then 1 else 0 end) as PullRequestEvent_count,
      SUM(CASE WHEN type = 'PullRequestReviewCommentEvent' then 1 else 0 end) as PullRequestReviewCommentEvent_count,
      SUM(CASE WHEN type = 'PushEvent' then 1 else 0 end) as PushEvent_count,
      SUM(CASE WHEN type = 'ReleaseEvent' then 1 else 0 end) as ReleaseEvent_count,
      SUM(CASE WHEN type = 'WatchEvent' then 1 else 0 end) as WatchEvent_count,

      FROM 
      [githubarchive:year.2018]
    WHERE (actor.login = '""")

    end = ("""' AND public)
    GROUP by actor.login;

    """)

    nusers = len(user_list)
    for i, user in enumerate(user_list):
        if i < nusers - 1:
            txt += user + """' OR actor.login = '""" 
        else:
            txt += user2 + end
    return txt

In [139]:
import numpy as np
users = np.asarray(contrib)[:, 0]

In [145]:
import pandas as pd
user_df = pd.DataFrame(contrib, columns=['login', 'repo_contributions_count'])

,login,repo_contributions_count
0,charris,4094
1,teoliphant,2065
2,cournape,1525
3,pearu,1061
4,eric-wieser,866
5,pv,762
6,mwiebe,759
7,juliantaylor,745
8,rgommers,586
9,stefanv,449


In [140]:
query = create_query_from_list(users)

In [141]:
job_config = bigquery.QueryJobConfig()
job_config.use_legacy_sql = True
df = client.query(query, location="US", job_config=job_config).to_dataframe()
df

,actor_login,event_count,last_event,first_event,CommitCommentEvent_count,CreateEvent_count,DeleteEvent_count,ForkEvent_count,GollumEvent_count,IssueCommentEvent_count,IssuesEvent_count,MemberEvent_count,PublicEvent_count,PullRequestEvent_count,PullRequestReviewCommentEvent_count,PushEvent_count,ReleaseEvent_count,WatchEvent_count
0,hadley,10734,2019-01-05 23:18:47+00:00,2018-02-01 04:49:54+00:00,50,155,58,8,0,3770,2088,11,1,988,1134,2441,30,0
1,chrisdembia,2554,2018-12-30 14:08:20+00:00,2018-02-01 00:01:06+00:00,38,135,105,7,0,1027,221,0,0,211,264,426,3,117
2,jakirkham,20484,2019-01-05 18:43:32+00:00,2018-02-01 00:03:25+00:00,54,1281,1129,95,0,6547,1300,2,0,2323,1592,6089,33,39
3,rainwoodman,2360,2019-01-05 10:32:00+00:00,2018-02-01 01:32:43+00:00,14,175,104,17,0,467,93,5,0,217,175,1089,2,2
4,nils-werner,301,2019-01-04 17:50:37+00:00,2018-02-09 10:21:56+00:00,2,20,6,16,0,57,20,0,0,19,0,152,0,9
5,Anner-deJong,162,2018-12-02 15:09:43+00:00,2018-02-01 17:19:53+00:00,0,16,0,7,0,6,3,0,0,9,4,111,0,6
6,pizzathief,24,2018-08-07 22:57:58+00:00,2018-06-29 06:21:02+00:00,0,0,0,1,0,0,0,0,0,15,0,8,0,0
7,solarjoe,40,2019-01-05 08:31:33+00:00,2018-02-06 17:33:06+00:00,0,1,0,1,0,19,11,0,0,2,0,2,0,4
8,ajschol,7,2018-11-11 03:48:58+00:00,2018-02-12 06:53:29+00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,7
9,bennorth,24,2018-12-15 21:46:49+00:00,2018-02-26 21:33:16+00:00,0,5,0,1,0,4,1,0,0,1,1,10,0,1


In [152]:
count_columns =  ['CommitCommentEvent_count', 'CreateEvent_count', 'DeleteEvent_count',
                  'ForkEvent_count', 'GollumEvent_count', 'IssueCommentEvent_count',
                  'IssuesEvent_count', 'MemberEvent_count', 'PublicEvent_count', 
                  'PullRequestEvent_count', 'PullRequestReviewCommentEvent_count',
                  'PushEvent_count', 'ReleaseEvent_count', 'WatchEvent_count']

In [159]:
contributors_df = pd.DataFrame(contrib, columns=['login', 'repo_contributions_count'])
contributors_df = contributors_df.merge(df, right_on='actor_login', left_on='login', how='left')
contributors_df[count_columns] = contributors_df[count_columns].fillna(0).astype(int)
contributors_df

,login,repo_contributions_count,actor_login,event_count,last_event,first_event,CommitCommentEvent_count,CreateEvent_count,DeleteEvent_count,ForkEvent_count,GollumEvent_count,IssueCommentEvent_count,IssuesEvent_count,MemberEvent_count,PublicEvent_count,PullRequestEvent_count,PullRequestReviewCommentEvent_count,PushEvent_count,ReleaseEvent_count,WatchEvent_count
0,charris,4094,charris,5361.0,2019-01-05 22:45:35+00:00,2018-02-01 15:01:45+00:00,13,231,227,0,2,2364,217,0,0,904,486,906,11,0
1,teoliphant,2065,teoliphant,110.0,2019-01-03 23:59:19+00:00,2018-02-21 17:14:48+00:00,1,11,1,2,5,47,5,5,0,8,0,17,0,8
2,cournape,1525,cournape,14.0,2018-11-02 08:09:16+00:00,2018-04-21 16:47:14+00:00,0,4,0,0,0,3,0,0,0,0,0,6,0,1
3,pearu,1061,pearu,732.0,2019-01-02 17:25:10+00:00,2018-02-12 17:18:49+00:00,0,22,1,0,0,102,18,0,0,20,141,428,0,0
4,eric-wieser,866,eric-wieser,5580.0,2019-01-05 14:54:23+00:00,2018-02-01 00:38:01+00:00,30,187,28,7,0,2001,177,0,0,321,2273,548,0,8
5,pv,762,pv,3741.0,2019-01-03 16:10:40+00:00,2018-02-01 06:55:24+00:00,1,177,197,2,35,891,195,0,0,384,122,1731,6,0
6,mwiebe,759,mwiebe,8.0,2018-09-03 01:41:51+00:00,2018-08-11 05:08:10+00:00,0,1,0,1,0,3,0,0,0,1,0,1,0,1
7,juliantaylor,745,juliantaylor,299.0,2019-01-02 18:55:44+00:00,2018-02-23 09:11:46+00:00,1,22,11,8,0,130,25,0,0,26,22,52,0,2
8,rgommers,586,rgommers,3208.0,2019-01-05 23:51:40+00:00,2018-02-01 05:37:04+00:00,0,95,74,2,8,1580,233,1,0,390,421,403,1,0
9,stefanv,449,stefanv,3281.0,2019-01-05 22:30:37+00:00,2018-02-01 18:14:45+00:00,0,95,7,9,6,592,80,7,0,155,206,2094,0,30


In [2]:
h.get_repo_contrib_history(gh, client, 'numpy', 'numpy')

UnboundLocalError: local variable 'count_columns' referenced before assignment